In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

Consider simple Lindblad master equation with two dissipators:

$m
     \frac{d\rho}{dt} = - \frac{i}{\hbar}[H, \rho]+\gamma_1\left[J_1 \rho J_1^\dagger - \frac{1}{2}\left\{ J_1^\dagger J_1, \rho \right\} \right]
     +\gamma_2\left[J_2 \rho J_2^\dagger - \frac{1}{2}\left\{ J_2^\dagger J_2, \rho \right\} \right]
$,

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = \begin{pmatrix} 0 & r \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$


In [7]:
# Define polynomial variables
@polyvar w r 

Hˢʸᵐᵇ = [ w   0
          0   0. ]

J₁ˢʸᵐᵇ = [ 0   r
           0   0. ]


function g_objective1(γᵢ)

    objₑₓ = 0

    for df_trn in train_files # loop over initial states

        ρᵗʳⁿ, tᵗʳⁿ = LiPoSID.get_rho_series(data_dir*df_trn*"_2CUT_data.h5", γᵢ)
        
        if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, [J₁ˢʸᵐᵇ])
        
    end # of files (initial states) loop

    return(objₑₓ)
end

g_objective1 (generic function with 1 method)

In [8]:

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end

local_rand_min (generic function with 1 method)

In [9]:
data_dir = "../DATA/"
println(data_dir)

models_dir = "../MODELS/"
tests_dir = "../TESTS/"

dodeca_files = ["State_D"*string(n) for n=1:20];

basis_files = ["State_B"*string(n) for n=1:4];

all_files = vcat(dodeca_files, basis_files)
train_files = basis_files 
test_files = dodeca_files;


../DATA/


In [10]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_data_file_name = "POP_LME_1_random_trn4_tst20_"*date_and_time_string * ".h5"

for γᵢ in γ

    println("γ =  "*γᵢ)

    poly = g_objective1(γᵢ)
    
    sol = local_rand_min(poly)

    omega = subs(w, sol)
    relaxation = subs(r^2, sol)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)
    J₁ˢⁱᵈ = subs(J₁ˢʸᵐᵇ, sol)


    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ) # create coupling group
        γ_group["omega"] = convert(Float64, omega)
        γ_group["gamma_relaxation"] = convert(Float64, relaxation)
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)        
    end

    for df in test_files # loop over initial states
        
        print(df*" ")

        start_time = time()

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_2CUT_data.h5", γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈ), [DenseOperator(basis,J₁ˢⁱᵈ)])

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]                    

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group

            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            
        end
                    
        println(min(Fᴸᴹᴱₑₓ))
    
    end

    println()
end

println(tests_data_file_name)

γ =  0.079477


State_D1 0.9993686180193304


State_D2 

0.9993718454779247
State_D3 0.9993717277804646


State_D4 

...

0.9991036749641103
State_D7 0.9925934960314522
State_D8 0.9957393886187949
State_D9 0.99855653926169
State_D10 0.9992664886141376
State_D11 0.9965616147199606
State_D12 

0.9990294707075051
State_D13 0.9960157307589685
State_D14 0.998594942995025
State_D15 0.9993414241490322
State_D16 0.9993414241489653
State_D17 0.9925934960310894
State_D18 

0.9957393886187049
State_D19 0.99926648861414
State_D20 0.998556539261612

γ =  251.33
State_D1 

0.9965356821326405
State_D2 0.9951092850034878
State_D3 0.9957899341841885
State_D4 0.9944258961178565
State_D5 0.9934629396417418
State_D6 0.9934629396417418
State_D7 0.9996798923882488
State_D8 0.9982727163860567
State_D9 0.9947920896443649
State_D10 0.9936328461526378
State_D11 

0.9957899341841767
State_D12 0.9944258961178565
State_D13 0.9965356821326405
State_D14 0.9951092850035501
State_D15 0.9930190594864469
State_D16 0.9930190594863993
State_D17 0.9996798923882584
State_D18 0.9982727163860524
State_D19 0.9936328461526378
State_D20 0.9947920896442558

POP_LME_1_random_trn4_tst20_2024-Jan-21_at_16-18.h5
